# 09线性回归模型的pytorch简洁实现

## 1.导入pytorch框架，并确认其版本

In [1]:
import torch
from torch import nn
import numpy as np
torch.manual_seed(1)

In [2]:
print(torch.__version__)
torch.set_default_tensor_type('torch.FloatTensor')

1.3.1


## 2.生成数据集
在这里生成数据集跟之前的实现中是完全一样的。其中features是训练数据特征,labels是标签。

In [3]:
num_inputs = 2
num_examples = 1000

true_w = [2, -3.4]
true_b = 4.2

features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

## 3.读取数据集
PyTorch提供了data包来读取数据。由于data常用作变量量名,我们将导入的data模块用用Data代替。在每一一次迭代中,我们将随机读取包含10个数据样本的小批量样本。

In [4]:
import torch.utils.data as Data

batch_size = 10

# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)

# 随机读取小批量样本
data_iter = Data.DataLoader(
    dataset=dataset,            # torch的TensorDataset格式
    batch_size=batch_size,      # mini batch size
    shuffle=True,               # 是否随机选择数据
    num_workers=2,              # 在多线程中读取数据
)

这里data_iter的使用跟上一节中一样,让我们读取并输出第一个小批量数据样本。

In [5]:
for X, y in data_iter:
    print(X, '\n', y)
    break

tensor([[-0.1937,  0.2852],
        [-1.1784,  0.0629],
        [-0.1066, -0.8491],
        [-0.9084,  1.7490],
        [ 1.5382,  1.3554],
        [ 1.5827,  0.0061],
        [-2.3566,  0.3380],
        [ 0.3400,  1.0401],
        [ 1.8608,  0.8541],
        [ 0.1557,  1.9575]]) 
 tensor([ 2.8509,  1.6226,  6.8792, -3.5634,  2.6615,  7.3295, -1.6684,  1.3640,
         5.0219, -2.1364])


## 4.定义模型

首先,导入torch.nn模块。实际上,“nn”是neural networks(神经网网络)的缩写。该模块定义了大量神经网络层。之前我们已经使用过了autograd,而nn就是利用autograd来定义模型。

nn的核心数据结构是Module,它是一个抽象概念,既可以表示神经网络中的某个层(layer),也可以表示一个包含很多层的神经网络。

在实际使用中,最常见的做法是继承nn.Module,撰写自己的网络或者是层。

一个nn.Module实例应该包含一些层以及返回输出的前向传播(forward)方法。

下面先来用nn.Module实现一个线性回归模型。

In [6]:
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()      # 调用父函数进行初始化
        self.linear = nn.Linear(n_feature, 1)  
        # 函数原型：torch.nn.Linear(in_features, out_features, bias=True)
    
    # forward 定义前向传播
    def forward(self, x):
        y = self.linear(x)
        return y
    
net = LinearNet(num_inputs)
print(net)   # 使用print打印出网络的结构

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


我们还可以用nn.Sequential来更加方便地搭建网络,

Sequential是一个有序的容器,网络层将按照传入Sequential的顺序依次被添加到计算图中。

In [7]:
# 建立多层网络的方法
# 方法一
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传入其他层
    )

# 方法二
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
# net.add_module ......

# 方法三
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
          ('linear', nn.Linear(num_inputs, 1))
          # ......
        ]))

print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


可以通过net.parameters()来查看模型所有的可学习参数,此函数将返回一个生成器。

In [8]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[0.5347, 0.7057]], requires_grad=True)
Parameter containing:
tensor([0.6873], requires_grad=True)


作为一个单层神经网络,线性回归输出层中的神经元和输入层中各个输入完全连接。

因此,线性回归的输出层又叫全连接层。

## 5.初始化模型参数

在使用net前,我们需要初始化模型参数,如线性回归模型中的权重和偏差。

PyTorch在 init 模块中提供了多种参数初始化方法。这里的init是initializer的缩写。

我们通过init.normal_将每个权重参数初始化为随机采样于均值为0、标准差为0.01的正态分布。偏差会初始化为零。

In [9]:
from torch.nn import init

init.normal_(net[0].weight, mean=0.0, std=0.01)
init.constant_(net[0].bias, val=0.0)  # 或者可以使用net[0].bias.data.fill_(0)直接对bias的data进行修改

Parameter containing:
tensor([0.], requires_grad=True)

## 6.定义损失函数

PyTorch在nn模块中提供了各种损失函数,这些损失函数可看作是一种特殊的层,

PyTorch也将这些损失函数实现为nn.Module的子类。

我们现在使用它提供的均方误差损失作为模型的损失函数。

In [10]:
loss = nn.MSELoss()    # nn内置平方损失函数
# 函数原型：`torch.nn.MSELoss(size_average=None, reduce=None, reduction='mean')`

## 7.定义优化函数

同样,我们也无须自己实现小批量随机梯度下降算法。

torch.optim模块提供了很多常用的优化算法,比如SGD、Adam和RMSProp等。

下面我们创建一个用于优化net所有参数的优化器实例,并指定学习率为0.03的小批量随机梯度下降(SGD)为优化算法。

In [11]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)   # 内置随机梯度下降功能
# 函数原型：torch.optim.SGD(params, lr=, momentum=0, dampening=0, weight_decay=0, nesterov=False)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


## 8.训练模型

在使用Gluon训练模型时,我们通过调用optim实例的step函数来迭代模型参数。

按照小批量随机梯度下降的定义,我们在step函数中指明批量大小,从而对批量中样本梯度求平均。

In [12]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零,等价于net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.000336
epoch 2, loss: 0.000131
epoch 3, loss: 0.000086


我们分别比较学到的模型参数和真实的模型参数的值。

我们从net中获得需要的层,并访问其权重(weight)和偏差(bias)。

得到的参数和真实的参数很接近。

In [13]:
# 结果比较
dense = net[0]
print(true_w, dense.weight.data)
print(true_b, dense.bias.data)

[2, -3.4] tensor([[ 2.0003, -3.3994]])
4.2 tensor([4.2001])


## 总结
使用pytorch能够更加快速简洁地完成模型的设计与实现

torch.utils.data模块提供了有关数据处理的工具;

torch.nn模块定义了大量神经网络的层;

torch.nn.init模块定义了各种初始化方法;

torch.optim模块提供了模型参数初始化的各种方法。